In [24]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import numpy as np
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import requests
from PIL import Image
from io import BytesIO

def scrape_table(table):
    columns = []
    for column in table.find_elements(By.XPATH, './/th'):
        columns.append(column.text)

    # find all the rows of the table
    rows = table.find_elements(By.XPATH, './/tr')
    data = []
    # loop through each row
    for row in rows:
        # find all the cells in the row
        cells = row.find_elements(By.XPATH,'.//td')
        # initialize an empty list to store the cell data
        row_data = []
        # loop through each cell in the row
        for cell in cells:
            try:
                span = cell.find_element(By.XPATH, './/span')
                row_data.append(span.text)
            except NoSuchElementException:
                row_data.append("")
        data.append(row_data)

    df = pd.DataFrame(data)
    df = df.dropna()
    # drop columns with empty strings in all rows
    df = df.loc[:, ~(df == '').all(axis=0)]
    df.columns = columns
    df = df.reset_index(drop=True)
    
    return df

driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://esearch.delhigovt.nic.in/Complete_search.aspx')

# Find the dropdown elements by XPath and Use the Select class to interact with the dropdowns
SRO = Select(driver.find_element(By.XPATH, '//select[@id="ctl00_ContentPlaceHolder1_ddl_sro_s"]'))
SRO.select_by_visible_text('Central -Asaf Ali (SR III)')

year = Select(driver.find_element(By.XPATH, '//select[@id="ctl00_ContentPlaceHolder1_ddl_year_s"]'))
try:
    year.select_by_visible_text('2021-2022')
except StaleElementReferenceException:
    driver.refresh()
    year = Select(driver.find_element(By.XPATH, '//select[@id="ctl00_ContentPlaceHolder1_ddl_year_s"]'))
    year.select_by_visible_text('2021-2022')

locality_dropdown = driver.find_element(By.XPATH, '//select[@id="ctl00_ContentPlaceHolder1_ddl_loc_s"]')
locality = Select(locality_dropdown)

options = locality_dropdown.find_elements(By.XPATH, ".//option")

options1 = []
for option in options:
    if not option.text.endswith('*'):
        options1.append(option.text)

options1 = options1[1:]

for i in range(len(options1)):
    locality.select_by_visible_text(options1[i])
    
    # Capture the captcha image and manually solve it
    captcha_element = driver.find_element(By.XPATH, "//img[@alt='Captcha']")
    captcha_image_url = captcha_element.get_attribute('src')
    # Download the captcha image using the requests library
    import requests
    captcha_image_content = requests.get(captcha_image_url).content
    # Save the captcha image to a file
    with open('captcha_image.jpg', 'wb') as f:
        f.write(captcha_image_content)
    # Manually solve the captcha and enter the text into the appropriate field on the website
    captcha_text = input("Enter the captcha text: ")
    captcha_field = driver.find_element(By.XPATH, "//input[@name='ctl00$ContentPlaceHolder1$txtcaptcha_s']")
    captcha_field.send_keys(captcha_text)
        
    time.sleep(3)

    # Find the search button and click it
    search_button = driver.find_element(By.XPATH, "//input[@id='ctl00_ContentPlaceHolder1_btn_search_s']")
    search_button.click()
    
    time.sleep(3)
    
    try:    
        # find the table element by XPath
        table = driver.find_element(By.XPATH, '//table[@class="table table-responsive"]/tbody')
    except NoSuchElementException:
        driver.get('https://esearch.delhigovt.nic.in/Complete_search.aspx')

        # Find the dropdown elements by XPath and Use the Select class to interact with the dropdowns
        SRO = Select(driver.find_element(By.XPATH, '//select[@id="ctl00_ContentPlaceHolder1_ddl_sro_s"]'))
        SRO.select_by_visible_text('Central -Asaf Ali (SR III)')

        year = Select(driver.find_element(By.XPATH, '//select[@id="ctl00_ContentPlaceHolder1_ddl_year_s"]'))
        try:
            year.select_by_visible_text('2021-2022')
        except StaleElementReferenceException:
            driver.refresh()
            year = Select(driver.find_element(By.XPATH, '//select[@id="ctl00_ContentPlaceHolder1_ddl_year_s"]'))
            year.select_by_visible_text('2021-2022')
            
        locality_dropdown = driver.find_element(By.XPATH, '//select[@id="ctl00_ContentPlaceHolder1_ddl_loc_s"]')
        locality = Select(locality_dropdown)
        continue
    except UnexpectedAlertPresentException:
        # handle the alert
        alert = driver.switch_to.alert
        alert.accept()

    # scrape the first page of the table
    df = scrape_table(table)

    already_scraped = False
    
        # loop through the pages and scrape each page
    while True:
        try:
                # find the next button and click it
                next_button = driver.find_element(By.XPATH, '//input[@title="Next Page"]')
                driver.execute_script("arguments[0].scrollIntoView();", next_button)
                next_button.click()
        
                # wait for the table to load
                wait = WebDriverWait(driver, 10)
                table = wait.until(EC.presence_of_element_located((By.XPATH, '//table[@class="table table-responsive"]/tbody')))
        
                # scrape the table on the current page
                new_df = scrape_table(table)
        
                # concatenate the new data with the old data
                df = pd.concat([df, new_df], ignore_index=True)
        
                # wait for a few seconds to allow the page to load
                time.sleep(3) 
        
                already_scraped = True
                
                break
                
        except NoSuchElementException:
            alread_scraped = True
            break
            
    directory = 'C:/Users/HIMANGI/Y2C'
    
    # save Excel files to the directory
    df.to_excel(os.path.join(directory, options1[i] + '.xlsx'))
                
    driver.get('https://esearch.delhigovt.nic.in/Complete_search.aspx')

    # Find the dropdown elements by XPath and Use the Select class to interact with the dropdowns
    SRO = Select(driver.find_element(By.XPATH, '//select[@id="ctl00_ContentPlaceHolder1_ddl_sro_s"]'))
    SRO.select_by_visible_text('Central -Asaf Ali (SR III)')

    year = Select(driver.find_element(By.XPATH, '//select[@id="ctl00_ContentPlaceHolder1_ddl_year_s"]'))
    try:
        year.select_by_visible_text('2021-2022')
    except StaleElementReferenceException:
        driver.refresh()
        year = Select(driver.find_element(By.XPATH, '//select[@id="ctl00_ContentPlaceHolder1_ddl_year_s"]'))
        year.select_by_visible_text('2021-2022')
            
    locality_dropdown = driver.find_element(By.XPATH, '//select[@id="ctl00_ContentPlaceHolder1_ddl_loc_s"]')
    locality = Select(locality_dropdown)

Enter the captcha text: LG9LT
Enter the captcha text: FC49L
Enter the captcha text: 4AZYJ


In [173]:
import pandas as pd
import os

os.chdir('C:/Users/HIMANGI/Y2C')
file_list = os.listdir()

In [165]:
import re

# Define a function to convert area values to square meters
def convert_to_sqm(area):
    match = re.match(r'^([\d\.]+) (.+)$', area)
    if not match:
        return None
    value = float(match.group(1))
    unit = match.group(2).lower()
    if unit in ['sq. feet', 'sqft', 'sq ft', 'square feet']:
        return value / 10.764
    elif unit in ['sq. yard', 'sqyrd', 'sq yd', 'square yard']:
        return value / 1.196
    elif unit in ['sq. meter', 'sqm', 'sq m', 'square meter']:
        return value
    elif unit in ['acre']:
        return value * 4046.856
    elif unit in ['hectare', 'ha']:
        return value * 10000
    else:
        return None

In [168]:
for i in range(0,70):
    data = pd.read_excel(file_list[i], engine='openpyxl')
    data = data.drop('Unnamed: 0',axis=1)
    data['Area in Sq. Meter'] = data['Area'].apply(convert_to_sqm) 
    data.to_excel(file_list[i])

In [177]:
data = pd.read_excel(file_list[2], engine='openpyxl')
data.dtypes

Unnamed: 0             int64
Reg.No                 int64
Reg.Date              object
First Party           object
Second Party          object
Property Address      object
Area                  object
Deed Type             object
Property Type         object
Area in Sq. Meter    float64
dtype: object

In [190]:
import os
import pandas as pd

# Specify input and output directories
input_dir = 'C:/Users/HIMANGI/Y2C'
output_dir = 'C:/Users/HIMANGI/Y2C'

# Loop through all files in input directory
for filename in os.listdir(input_dir):
    # Check if file is an Excel file
    if filename.endswith('.xlsx'):
        # Read Excel file into pandas DataFrame
        filepath = os.path.join(input_dir, filename)
        data = pd.read_excel(filepath, engine='openpyxl')
        
        # Create output file path and save DataFrame as CSV
        output_filename = filename.replace('.xlsx', '.csv')
        output_filepath = os.path.join(output_dir, output_filename)
        data.to_csv(output_filepath, index=False)

In [191]:
import os

directory = 'C:/Users/HIMANGI/Y2C'

for filename in os.listdir(directory):
    if filename.endswith(".xlsx"):
        os.remove(os.path.join(directory, filename))
